In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
religion_unlab = pd.read_csv('AllBooks_baseline_DTM_Unlabelled.csv')
religion_lab = pd.read_csv('AllBooks_baseline_DTM_Labelled.csv')

print(religion_lab.shape)
religion_lab.head()

In [ ]:
religion_unlab.info()

In [ ]:
religion_lab['label'] = religion_lab['Unnamed: 0'].apply(lambda x: x.split('_')[0])

#for i in range(len(religion_lab)):
#    religion_lab['label'][i] = religion_lab['Unnamed: 0'][i].split('_')[0]
   

labels = religion_lab['label'].value_counts()
labels = pd.DataFrame(labels).reset_index()
labels.columns = ['label', 'count']

In [ ]:
fig = plt.figure(figsize = (9, 6))
sns.barplot(data = labels, y = 'label', x = 'count', color = 'dodgerblue')
fig.suptitle('Żródła tekstów', fontsize=18)
plt.xlabel('Liczba tesktów')
plt.ylabel("Księga")
plt.show()

In [ ]:
most_pop_words = religion_unlab.sum().sort_values(ascending=False).head(30)
most_pop_words = pd.DataFrame(most_pop_words).reset_index()
most_pop_words.columns = ['word', 'count']

fig = plt.figure(figsize = (9, 6))
sns.barplot(data = most_pop_words, x = 'word', y = 'count', color = 'dodgerblue')
plt.xticks(rotation=60)
fig.suptitle('Najpopularniejsze słowa', fontsize=18)
plt.xlabel('')
plt.ylabel('Liczba wystąpień we wszystkich tekstach')
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=100,
        max_font_size=30,
        scale=3,
        random_state=1)
   
    wordcloud=wordcloud.generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(religion_unlab.sum().sort_values(ascending=False).to_dict())

In [ ]:
religion_lab_stacked = religion_lab.drop('Unnamed: 0', axis = 1).groupby('label').sum().transpose()

fig, ax = plt.subplots(4, 2, figsize = (15, 24))
i = 0

for c in religion_lab_stacked.columns:
    df = pd.DataFrame(religion_lab_stacked[c].sort_values(ascending = False)).reset_index().head(10)
    df.columns = ['word', 'count']
    sns.barplot(data = df, x = 'word', y = 'count', color = 'dodgerblue', ax = ax[int(np.floor(i/2)%4), int(i%2)])
    ax[int(np.floor(i/2)%4), int(i%2)].set_title(c)
    ax[int(np.floor(i/2)%4), int(i%2)].set_xlabel('')
    ax[int(np.floor(i/2)%4), int(i%2)].set_xticklabels(ax[int(np.floor(i/2)%4), int(i%2)].get_xticklabels(), rotation=45)

    
    i+=1

In [ ]:
import re
from textstat import lexicon_count 
from textstat import flesch_reading_ease 
from textstat import flesch_kincaid_grade 
from textstat import sentence_count 
from textstat import lexicon_count 

In [ ]:
file = open('Complete_data .txt', 'r')
file_content = file.read()
file.close()

content_list = re.split('\d+\.\d+', file_content)

text = []
for i in range(len(content_list)):
    stripped = content_list[i].strip()
    if stripped != '':
        text.append(content_list[i])
        
substrings_to_drop = ['\n', ' \n', '\n ', '  \n', '§', '§ ']
            
for i in range(len(corpus)):
    for j in substrings_to_drop:
        corpus[i] = corpus[i].replace(j, '') 
        
data = pd.DataFrame(corpus).reset_index()
data.columns = ['index', 'text']

data.head()

In [ ]:
#characters
data['len'] = data['text'].str.len()
#words
data['words'] = data['text'].apply(lambda x : lexicon_count(x, removepunct=True))
#average sentence length
data['avg_sen'] = data['text'].str.split().apply(lambda x : [len(i) for i in x]).map(lambda x: np.mean(x))
#reading ease
data['reading_ease'] = data['text'].apply(lambda x : flesch_reading_ease(x))
#flesch_kincaid_grade
data['grade'] = data['text'].apply(lambda x : flesch_kincaid_grade(x))
#sentences
data['sentences'] = data['text'].apply(lambda x : sentence_count(x))

In [ ]:
data

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
sns.histplot(data = data, x = 'len', ax = ax1)
sns.histplot(data['words'], ax = ax2)
ax1.set_title('Rozkład liczby znaków w tekście')
ax2.set_title('Rozkład liczby słów w tekście')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
sns.histplot(np.log1p(data['sentences']), ax = ax1)
sns.histplot(data = data, x = 'avg_sen', ax = ax2)
ax1.set_title('Rozkład liczby zdać w tekstach (skala logarytmiczna)')
ax2.set_title('Rozkład średniej iczby zdań w tekstach')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
sns.histplot(data = data, x = 'reading_ease', ax = ax1)
sns.histplot(data['grade'], ax = ax2)
ax1.set_title('Flesh reading ease')
ax2.set_title('Flesh-Kincaid grade level')
plt.show()

## Druga część

In [ ]:
df = pd.read_csv("AllBooks_baseline_DTM_Unlabelled.csv")
print(f"Shape of data: {df.shape}.")

In [ ]:
df.head()

Mamy 8266 słów (kolumn), 590 rekordów. Wszystkie wartości są dodatnimi wartościami. Nie mamy braków w danych.

Na początek sprawdźmy czy mamy w naszej ramce danych tzw. skrótowce, czyli słówka typu "don't","aren't", isn't" itp.

In [ ]:
df.rename(columns = {"# foolishness":"foolishness"}, inplace = True)
for i in df.columns:
    if "'" in i: print(i)

Wniosek: nie mamy skrótowców, więc możemy pominąć punkt ich rozwijania. 

Z naszych słów wyciągnijmy korzeń. Może się zdarzyć, że mamy jednocześnie 2 różne formy tego samego wyrazu, np. 'play', 'playing', 'plays'. Dla naszego zadania jest to oczywiście jedno i to samo słowo. Zrobimy to ponownie wykorzystując bibliotekę Spacy.

In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()

listToStr = ' '.join([str(elem) for elem in df.columns])
doc = nlp(listToStr)

i=0
tokenDict = {}
for token in doc:
    if (str(token) != str(token.lemma_)): 
        tokenDict[str(token)] = token.lemma_
        
print(tokenDict)

In [ ]:
df.rename(columns = tokenDict, inplace = True)

In [ ]:
print(f"Kolumny unikalne: {len(df.columns.unique())}.")
print(f"Wszystkie kolumny: {len(df.columns)}.")
print("Przyklad recznie znaleziony zduplikowanych kolumn")
df["oppose"]

Pozbadzmy sie duplikujacych kolumn

In [ ]:
df = df.sum(axis=1, level=0)
print(df.shape)

Sprawdźmy teraz czy mamy słówka zaliczane do grupy 'najpopularniejszych słówek języka'. W języku angielskim są to słówka typu “the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc. Ponownie wykorzystamy bibliotekę spaCy.

In [ ]:
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS


nlp = English()
stopwords = []
for i in df.columns:
    lexeme = nlp.vocab[i]
    if lexeme.is_stop == True: stopwords.append(i)
print(stopwords)
print(len(stopwords))

In [ ]:
print(f"{len(stopwords)} słów z naszej ramki zostało zklasyfikowane jako słowa o niskiej wartości dla całościowego znaczenia tekstu. Spośród ponad 6000 wszystkich słów, stanowią one niewielki procent więc możemy je usunąć.")

In [ ]:
df = df.drop(columns = stopwords)
print(df.shape)

Teraz sprawdźmy najczęściej pojawiające się słówka

In [ ]:
most_pop_words = df.sum().sort_values(ascending=False).head(30)
most_pop_words = pd.DataFrame(most_pop_words).reset_index()
most_pop_words.columns = ['word', 'count']

fig = plt.figure(figsize = (9, 6))
sns.barplot(data = most_pop_words, x = 'word', y = 'count', color = 'dodgerblue')
plt.xticks(rotation=60)
fig.suptitle('Most popular words in whole dataset', fontsize=18)
plt.show()

print(f"Średnia liczba wystąpień jednego słowa: {np.mean(most_pop_words['count']).round(2)}")
print(f"Odchylenie standardowe liczby wystąpień jednego słowa: {np.std(most_pop_words['count']).round(2)}")

In [ ]:
d = {'word': df.columns}
word_len = pd.DataFrame(data = d)
word_len['nchars'] = word_len['word'].apply(lambda x: len(x))
word_len['occurences'] = word_len['word'].apply(lambda x: df[x].sum())


fig = plt.figure(figsize = (9, 6))
word_len['nchars'].plot(kind = 'hist', title = 'Rozkład długości wyrazów ze wszystkich tekstów', bins = 25
                           , xlabel = "Liczba liter", ylabel = 'Liczba słów o danej długości')
plt.show()
print(f"Średnia długość słowa: {np.mean(word_len['nchars']).round(2)}")
print(f"Odchylenie standardowe: {np.std(word_len['nchars']).round(2)}")

In [ ]:
# Sprawdźmy te słowa, które sa bardzo długie albo krótkie
short_words = word_len.loc[word_len['nchars'] == 2]
short_words = short_words['word'].to_numpy()

long_words = word_len.loc[word_len['nchars'] >= 17]
long_words = long_words['word'].to_numpy()

print(word_len.loc[word_len['nchars'] == 2])
print(word_len.loc[word_len['nchars'] >= 17])

In [ ]:
# Ponieważ liczby wystąpień tych słów są dużo niższe niż odchylenie standardowe, również usuniemy je z ramki danych

df = df.drop(columns = (short_words), axis = 1)
df = df.drop(columns = (long_words), axis = 1)

In [ ]:
df.shape

Oceńmy teraz czy nasze słowa są nacechowane pozytywnie czy negatywnie.

In [ ]:
from textblob import TextBlob

def polarity(text):
    return TextBlob(text).sentiment.polarity

def sentiment(x):
    if x<0:
        return 'neg'
    elif x==0:
        return 'neu'
    else:
        return 'pos'
    
def subjectivity(text):
    return TextBlob(text).sentiment.subjectivity
    
word_len['polarity_score']=word_len['word'].\
   apply(lambda x : polarity(x))

word_len['polarity']=word_len['polarity_score'].\
   map(lambda x: sentiment(x))

word_len['subjectivity']=word_len['word'].\
   map(lambda x: subjectivity(x))

fig, axs = plt.subplots(1, 2, figsize = (15, 6))
print("Nacechowanie emocjonalne słów:")
word_len['polarity_score'].hist(ax = axs[0])
word_len['polarity'].hist(ax = axs[1])

plt.show()

plot_sub = word_len['subjectivity'].hist()
plot_sub.set_title("Obiektywność")

Wniosek: większość słów z naszej bazy ma neutralne nacechowanie emocjonalne. Z pozostałych nielicznych słów, większość jest nacechowana pozytywnie. Słowa są również raczej obiektywne.